In [1]:
pip install mysql-connector-python            # to establish connection to MySQL database

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [103]:
pip install googletrans

In [3]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [68]:
import numpy as np
import pandas as pd
import mysql.connector
from googletrans import Translator

# 1. Data extraction & Exploration

### 1.1 Events Data

In [5]:
df_events = pd.read_csv(r"C:\Users\aksha\Downloads\events_data.csv")
df_events.head()

,event_id,device_id,timestamp,longitude,latitude,city,state
0,2765368,2.973348e+18,2016-05-07 22:52:05,77.225676,28.730140,Delhi,Delhi
1,2955066,4.734221e+18,2016-05-01 20:44:16,88.388361,22.660325,Calcutta,WestBengal
2,605968,-3.264500e+18,2016-05-02 14:23:04,77.256809,28.757906,Delhi,Delhi
3,448114,5.731369e+18,2016-05-03 13:21:16,80.343613,13.153332,Chennai,TamilNadu
4,665740,3.388880e+17,2016-05-06 03:51:05,85.997745,23.842609,Bokaro,Jharkhand


In [6]:
df_events.shape

(3252950, 7)

In [7]:
df_events.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252950 entries, 0 to 3252949
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   event_id   3252950 non-null  int64  
 1   device_id  3252497 non-null  float64
 2   timestamp  3252950 non-null  object 
 3   longitude  3252527 non-null  float64
 4   latitude   3252527 non-null  float64
 5   city       3252950 non-null  object 
 6   state      3252573 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 173.7+ MB


In [8]:
df_events.describe()

,event_id,device_id,longitude,latitude
count,3.252950e+06,3.252497e+06,3.252527e+06,3.252527e+06
mean,1.626476e+06,1.012200e+17,7.815868e+01,2.168851e+01
std,9.390459e+05,5.316758e+18,4.235570e+00,5.789111e+00
min,1.000000e+00,-9.222957e+18,1.256700e+01,8.190110e+00
25%,8.132382e+05,-4.540611e+18,7.583543e+01,1.780171e+01
50%,1.626476e+06,1.726820e+17,7.726814e+01,2.216454e+01
75%,2.439713e+06,4.861813e+18,8.031916e+01,2.868278e+01
max,3.252950e+06,9.222849e+18,9.545946e+01,4.187190e+01


__Observations__
- Events data has 3252950 rows and 7 colums
- Dtype for event_id and device_id to be changed to float and timestamp to be changed to datetime
- 435 missing values in Device_id and Latitude, longitude have 423 mising values whereas state has 337 missing value
- Need to fill missing states, lats and long


### 1.2 Datasets from MySQL

In [9]:
connection = mysql.connector.connect(host="cpanel.insaid.co", database = 'Capstone1',user="student", passwd="student")
df_gender_age = pd.read_sql('''Select * from gender_age_train;''',connection)
df_brand_model = pd.read_sql('''Select * from phone_brand_device_model;''',connection)
connection.close()

In [10]:
df_gender_age.head()

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [11]:
df_gender_age.shape

(74645, 4)

In [12]:
df_gender_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74645 entries, 0 to 74644
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   device_id  74645 non-null  int64 
 1   gender     74645 non-null  object
 2   age        74645 non-null  int64 
 3   group      74645 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


In [13]:
df_gender_age.describe()

,device_id,age
count,7.464500e+04,74645.000000
mean,-7.491354e+14,31.410342
std,5.327150e+18,9.868735
min,-9.223067e+18,1.000000
25%,-4.617367e+18,25.000000
50%,-1.841362e+16,29.000000
75%,4.636656e+18,36.000000
max,9.222849e+18,96.000000


__Observation__
- There are no null values in this data set
- Dtype for 'device_id' is to be changed to float

In [86]:
df_brand_model.head()

,device_id,phone_brand,device_model
0,1877775838486905855,vivo,Y13
1,-3766087376657242966,Xiaomi,V183
2,-6238937574958215831,OPPO,R7s
3,8973197758510677470,Samsung,A368t
4,-2015528097870762664,Xiaomi,红米Note2


In [85]:
df_brand_model.shape

(87726, 3)

In [16]:
df_brand_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87726 entries, 0 to 87725
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   device_id     87726 non-null  int64 
 1   phone_brand   87726 non-null  object
 2   device_model  87726 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


__Observation__
- There are no null values in this data set
- Dtype for 'device_id' is to be changed to float
- Certain phone brand names are in Chinese. These need to be translated to English

# 2 Data Cleaning

### 2.1 Missing states

In [17]:
df_events[df_events['state'].isnull()]['city'].unique()

array(['Pune', 'Visakhapatnam', 'Indore', 'Chennai', 'Delhi',
       'Channapatna', 'Jaipur', 'Gangarampur', 'Arambagh'], dtype=object)

In [18]:
#Replacing null states for `WestBengal` & Karnatak
df_events.loc[(df_events['city'] =='Arambagh') \
    & (df_events['state'].isnull()),'state'] = 'WestBengal'

df_events.loc[(df_events['city'] =='Gangarampur') \
    & (df_events['state'].isnull()),'state'] = 'WestBengal'

#Replacing null states only for `Karnataka`
df_events.loc[(df_events['city'] =='Channapatna') \
    & (df_events['state'].isnull()),'state'] ='Karnataka'

In [19]:
list_of_states = ['WestBengal', 'Karnataka', 'Gujarat', 'Bihar', 'Punjab', 'Kerala']

In [20]:
df_events_data = df_events.loc[df_events['state'].isin(list_of_states)]

In [21]:
df_events_data.shape

(422971, 7)

### 2.2 Missing Latitude & Longitude

In [22]:
df_events_data [df_events_data['longitude'].isnull()]['city'].unique()

array(['Araria', 'Moga', 'Bagaha'], dtype=object)

In [23]:
df_events_data[df_events_data['longitude'].isnull()]['state'].unique()

array(['Bihar', 'Punjab'], dtype=object)

In [24]:
Araria_latitude_mode = df_events_data['latitude'][df_events_data['city'] == 'Araria'].mode()
Araria_latitude_mode

0    26.211919
dtype: float64

In [25]:
Moga_latitude_mode = df_events_data['latitude'][df_events_data['city'] == 'Moga'].mode()
Moga_latitude_mode

0    30.904178
dtype: float64

In [26]:
Bagaha_latitude_mode = df_events_data['latitude'][df_events_data['city'] == 'Bagaha'].mode()
Bagaha_latitude_mode

0    27.177397
dtype: float64

In [27]:
Araria_longitude_mode = df_events_data['longitude'][df_events_data['city'] == 'Araria'].mode()
Araria_longitude_mode

0    87.570736
dtype: float64

In [28]:
Moga_longitude_mode = df_events_data['longitude'][df_events_data['city'] == 'Moga'].mode()
Moga_longitude_mode

0    75.268748
dtype: float64

In [29]:
Bagaha_longitude_mode = df_events_data['longitude'][df_events_data['city'] == 'Bagaha'].mode()
Bagaha_longitude_mode

0    84.1409
dtype: float64

In [30]:
#Replacing missing longitude & latitude in `Araria`,`Bagaha` & `Moga`
df_events_data.loc[(df_events_data['city'] =='Araria') & (df_events_data['longitude'].isnull()),'longitude']= '87.570736'
df_events_data.loc[(df_events_data['city'] =='Araria') & (df_events_data['latitude'].isnull()),'latitude']= '26.211919'

df_events_data.loc[(df_events_data['city'] =='Moga') & (df_events_data['longitude'].isnull()), 'longitude']= '75.268748'
df_events_data.loc[(df_events_data['city'] =='Moga') & (df_events_data['latitude'].isnull()),'latitude']= '30.904178'

df_events_data.loc[(df_events_data['city'] =='Bagaha') & (df_events_data['longitude'].isnull()), 'longitude']= ' 84.1409'
df_events_data.loc[(df_events_data['city'] =='Bagaha') & (df_events_data['latitude'].isnull()),'latitude']= '27.177397'

C:\Users\aksha\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


### 2.3 Missing devices

In [31]:
df_events_data[df_events_data['device_id'].isnull()]['city'].unique()

array(['Hoshiarpur', 'Bardoli', 'Jetpur'], dtype=object)

In [32]:
df_events_data[df_events_data['device_id'].isnull()].groupby(df_events_data['city']).count()

,event_id,device_id,timestamp,longitude,latitude,city,state
city,,,,,,,
Bardoli,16,0,16,16,16,16,16
Hoshiarpur,16,0,16,16,16,16,16
Jetpur,16,0,16,16,16,16,16


In [33]:
pd.options.display.float_format = '{:,.2f}'.format
df_events_data['device_id'][df_events_data['city']=='Jetpur'].mode()

0   -1,688,015,122,502,424,064.00
dtype: float64

In [34]:
pd.options.display.float_format = '{:,.2f}'.format
df_events_data['device_id'][df_events_data['city']=='Bardoli'].mode()

0   -8,215,770,519,233,685,504.00
dtype: float64

In [35]:
pd.options.display.float_format = '{:,.2f}'.format
df_events_data['device_id'][df_events_data['city']=='Hoshiarpur'].mode()

0   1,750,778,632,182,066,944.00
dtype: float64

In [36]:
#Replacing missing `device_id` in Jetpur, Bardoli and Hoshiarpur
df_events_data.loc[(df_events_data['city'] =='Jetpur') & (df_events_data['device_id'].isnull()),'device_id']= '-1,688,015,122,502,424,064'

df_events_data.loc[(df_events_data['city'] =='Bardoli') & (df_events_data['device_id'].isnull()), 'device_id']= '-8,215,770,519,233,685,504'

df_events_data.loc[(df_events_data['city'] =='Hoshiarpur') & (df_events_data['device_id'].isnull()), 'device_id']= '1,750,778,632,182,066,944'

C:\Users\aksha\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [37]:
list2 = ['Jetpur', 'Hoshiarpur', 'Bardoli']
df_events_data[df_events_data['city'].isin(list2)].groupby(df_events_data['city']).count()

,event_id,device_id,timestamp,longitude,latitude,city,state
city,,,,,,,
Bardoli,1475,1475,1475,1475,1475,1475,1475
Hoshiarpur,914,914,914,914,914,914,914
Jetpur,1479,1479,1479,1479,1479,1479,1479


In [38]:
df_events_data.isnull().sum()

event_id     0
device_id    0
timestamp    0
longitude    0
latitude     0
city         0
state        0
dtype: int64

### 2.4 Translation

In [64]:
!pip install google-trans-new

In [65]:

from google_trans_new import google_translator  
import google_trans_new

In [87]:
df_brand_model['phone_brand'].unique()

array(['vivo', 'Xiaomi', 'OPPO', 'Samsung', 'Coolpad', 'Lenovo', 'Huawei',
       'Qiku', 'Meizu', ' Feixun ', 'China Mobile', 'HTC', 'Amagatarai',
       'Extreme Treasure', 'LG', 'Obson', 'Youmi', 'ZUK', 'Nubia', 'HP',
       'Nibiru', 'Meitu', 'Xianmi', 'Motorola', 'Mengmi', 'Hammer',
       ' Focus', 'LeEco', 'Hisense', 'Bai Li Feng', 'One Plus', 'Letters',
       'Haier', ' Coolby ', 'Newman', 'Flyer', 'Duowei', 'Lingyun', 'TCL',
       'Cooper', 'Aipair', 'LOGO', 'Scallion', 'Fruit Rice', 'ASUS',
       'Onda', ' Aiyouni ', 'Kangjia', 'Yougo', 'Banghua', '赛博宇华',
       'Black Rice', 'Lovme', 'Pioneer', 'E-Pay', 'Shenzhou', 'Nokia',
       'Punale', ' Candied ', 'Yi Tong', 'Ou Xin', '米奇', 'Cubei Cube',
       'Blue Devils', 'Little Poplar', 'Bellfonte', 'Glutinous Rice',
       'Mi Ge', 'E人E本', 'Simi', 'Big Q', 'Taipower', 'Philips', 'Wei Mi',
       '大显', 'Changhong', 'Weitu', 'Green Orange', '本为', 'Shrimp', 'Amoi',
       '帷幄', '百加', 'SUGAR', 'Ouqi', '世纪星', '智镁', 'Obi', '基伍', '飞

In [88]:
brands_in_chinese = ['华为','小米','三星','魅族','酷派','乐视','联想 ','vivo','OPPO','HTC','LG','TCL','LOGO','Lovme','ZUK','PPTV','努比亚','海信','天语','夏新','欧比', '优米', '欧博信', '奇酷', '锤子', '酷比魔方', '语信', '朵唯', '德赛', '一加',
       '华硕', '普耐尔', '易派', '摩托罗拉', '优购', '酷珀', '梦米', '爱派尔', '昂达', '邦华',
       '聆韵', '中国移动', '诺亚信', '酷比', '艾优尼', '斐讯', '富可视', '欧新', '美图', '宝捷讯',
       '青橙', '果米', '波导', '诺基亚', '神舟', '糯米', '亿通', '纽曼', '唯米', '百立丰',
       '沃普丰', '虾米', '凯利通', '贝尔丰', '至尊宝', '蓝魔', '青葱', '白米', '大可乐', '尼比鲁',
       '康佳', '乡米', '黑米', '海尔', '优语', '维图', '米歌', '广信', 'E派', '小杨树', '糖葫芦',
       '西米', '谷歌', '长虹', '飞利浦', '惠普', '台电', '大Q', '鲜米', '先锋', '欧奇']
brands_in_english = ['Huawei','Xiaomi','Samsung','Meizu','Coolpad','LeEco','Lenovo','vivo','OPPO','HTC','LG','TCL','LOGO','Lovme','ZUK','PPTV','Nubia','Hisense','Amagatarai','Amoi','Obi', 'Youmi', 'Obson', 'Qiku', 'Hammer', 'Cubei Cube', 'Letters', 'Duowei', 'Desai', 'One Plus','ASUS', 'Punale', 'Yipai', 'Motorola', 'Yougo', 'Cooper', 'Mengmi', 'Aipair', 'Onda', 'Banghua', 'Lingyun', 'China Mobile', 'Noah',' Coolby ',' Aiyouni ',' Feixun ',' Focus', 'Ou Xin', 'Meitu', 'Projet News','Green Orange', 'Fruit Rice', 'Flyer', 'Nokia', 'Shenzhou', 'Glutinous Rice', 'Yi Tong', 'Newman', 'Wei Mi', 'Bai Li Feng','Wopfung', 'Shrimp', 'Kellystone', 'Bellfonte', 'Extreme Treasure', 'Blue Devils', 'Scallion', 'White Rice', 'Big Cola', 'Nibiru','Kangjia', 'Xianmi', 'Black Rice', 'Haier', 'Youyu', 'Weitu', 'Mi Ge', 'Guangxin', 'E-Pay', 'Little Poplar', ' Candied ','Simi', 'Google', 'Changhong', 'Philips', 'HP', 'Taipower', 'Big Q', 'Fresh Rice', 'Pioneer', 'Ouqi']

brands_dict1 = dict(zip(brands_in_chinese, brands_in_english)) 

In [89]:
def translator(val):
    if val in brands_dict1:
        return brands_dict1[val]
    else:
        return val

In [90]:
df_brand_model['phone_brand'] = df_brand_model['phone_brand'].apply(translator)

In [92]:
df_brand_model.head()

,device_id,phone_brand,device_model
0,1877775838486905855,vivo,Y13
1,-3766087376657242966,Xiaomi,V183
2,-6238937574958215831,OPPO,R7s
3,8973197758510677470,Samsung,A368t
4,-2015528097870762664,Xiaomi,红米Note2


Merging Data

In [105]:
df_gender_age['device_id']=df_gender_age['device_id'].astype(float)

In [106]:

df_brand_model['device_id']=df_brand_model['device_id'].astype(float)

In [107]:
df_events_data['device_id']=df_events_data['device_id'].astype(float)

ValueError: could not convert string to float: '1,750,778,632,182,066,944'

In [110]:
df_final = df_events_data.merge(df_brand_model,on='device_id').merge(df_gender_age,on='device_id')


ValueError: You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat